<a href="https://colab.research.google.com/github/kairamilanifitria/Bootcamp-NLP/blob/main/09092024_POS_Tagging_%26_Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sesi 13

# POS Tagging

Pada pembuatan POS Tagging kali ini, Algoritma yang akan digunakan adalah Random Forest

In [1]:
!pip install nlp_id  # untuk kepeluan tokenizer, bukan postag

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 15.4 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=440b2f88dbb6e2ec89b11d3f1829576eefacfa56e47fd05d34f26b053ea382e1
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
  Attempting uninstall: huggingface-hub
    Found ex

In [2]:
import nltk
import os
import pickle
import warnings
import wget
from nlp_id.tokenizer import Tokenizer
from nltk.tree import Tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
tokenizer = Tokenizer()

In [5]:
dataset_path = "/content/drive/MyDrive/Bootcamp AI/Dataset/dataset_postag.txt"

In [6]:
def read_dataset(dataset_path=None):

    with open(dataset_path) as f:
        raw_file = f.read().split("\n")

    files = [i.split("\t") for i in raw_file]

    sentences, tags, temp_sentences, temp_tags = [], [], [], []

    for file in files:
        if file != [""]:
            temp_sentences.append(file[0])  # get the sentences
            temp_tags.append(file[1])  # get the tag
        else:
            # check if the temp sentences and temp tags is not null
            # and both of them have the same length
            if len(temp_sentences) > 0 and (
                len(temp_sentences) == len(temp_tags)
            ):
                sentences.append(temp_sentences)
                tags.append(temp_tags)
            temp_sentences, temp_tags = [], []
    return sentences, tags

In [7]:
read_dataset(dataset_path)

([['Ditjen',
   'Bea',
   'dan',
   'Cukai',
   'Kementerian',
   'Keuangan',
   '(',
   'Kemenkeu',
   ')',
   'memastikan',
   'ada',
   'nya',
   '18',
   'kotak',
   'selundupan',
   'yang',
   'dibawa',
   'melalui',
   'pesawat',
   'baru',
   'Airbus',
   'A330-900',
   'Neo',
   'milik',
   'Garuda',
   'Indonesia',
   '.'],
  ['Pengambilan',
   'foto',
   'dilakukan',
   'dari',
   'kepala',
   'perahu',
   'sehingga',
   'menghasilkan',
   'foto',
   'yang',
   'sangat',
   'menarik',
   '.'],
  ['Vice',
   'President',
   'Corporate',
   'Secretary',
   'Garuda',
   'Indonesia',
   ',',
   'M',
   '.'],
  ['Di',
   'sana',
   ',',
   'penumpang',
   'mengungkapkan',
   'atau',
   'men-declare',
   'barang',
   'bawaan',
   ',',
   'termasuk',
   'yang',
   'disebut',
   'sparepart',
   'Harley',
   'dan',
   'sepeda',
   'Brompton',
   '.'],
  ['Kita', '(', 'penumpang', ')', 'self', 'declare', '.'],
  ['Untuk',
   'clean',
   'and',
   'jerk',
   ',',
   'sosok',
   'berusia

In [8]:
sentences, tags = read_dataset(dataset_path)

In [9]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        "word": sentence[index],
        "is_first": index == 0,
        "is_last": index == len(sentence) - 1,
        "is_capitalized": sentence[index][0].upper()
        == sentence[index][0],
        "is_all_caps": sentence[index].upper() == sentence[index],
        "is_all_lower": sentence[index].lower() == sentence[index],
        "has_hyphen": "-" in sentence[index],
        "is_numeric": sentence[index].isdigit(),
        "capitals_inside": sentence[index][1:].lower()
        != sentence[index][1:],
        "prefix-1": sentence[index][0],
        "prefix-1-lower": sentence[index][0].lower(),
        "prefix-2": sentence[index][:2],
        "prefix-2-lower": sentence[index][:2].lower(),
        "prefix-3": sentence[index][:3],
        "prefix-3-lower": sentence[index][:3].lower(),
        "suffix-1": sentence[index][-1],
        "suffix-1-lower": sentence[index][-1].lower(),
        "suffix-2": sentence[index][-2:],
        "suffix-2-lower": sentence[index][-2:].lower(),
        "suffix-3": sentence[index][-3:],
        "suffix-3-lower": sentence[index][-3:].lower(),
        "lowercase_word": sentence[index].lower(),
        "prev_word": "" if index == 0 else sentence[index - 1],
        "next_word": ""
        if index == len(sentence) - 1
        else sentence[index + 1],
        "prev_word_is_capitalized": False
        if index == 0
        else sentence[index - 1][0].upper() == sentence[index - 1][0],
        "next_word_is_capitalized": False
        if index == len(sentence) - 1
        else sentence[index + 1][0].upper() == sentence[index + 1][0],
        "2-prev-word": "" if index <= 1 else sentence[index - 2],
        "2-next-word": ""
        if index >= len(sentence) - 2
        else sentence[index + 2],
    }

In [10]:
sentences[0]

['Ditjen',
 'Bea',
 'dan',
 'Cukai',
 'Kementerian',
 'Keuangan',
 '(',
 'Kemenkeu',
 ')',
 'memastikan',
 'ada',
 'nya',
 '18',
 'kotak',
 'selundupan',
 'yang',
 'dibawa',
 'melalui',
 'pesawat',
 'baru',
 'Airbus',
 'A330-900',
 'Neo',
 'milik',
 'Garuda',
 'Indonesia',
 '.']

In [11]:
def transform_to_dataset(sentences, tags):
    X, y = [], []

    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])

    return X, y

In [12]:
clf = Pipeline(
        [
            ("vectorizer", DictVectorizer(sparse=True)),
            (
                "classifier",
                RandomForestClassifier(
                    criterion="gini", n_estimators=15, random_state=2020
                ),
            ),
        ]
    )

In [13]:
def train(sentences, tags):
    """
    training
    """
    clf.fit(sentences, tags)

In [14]:
sentences, tags = transform_to_dataset(sentences, tags)

In [16]:
sentences

[{'word': 'Ditjen',
  'is_first': True,
  'is_last': False,
  'is_capitalized': True,
  'is_all_caps': False,
  'is_all_lower': False,
  'has_hyphen': False,
  'is_numeric': False,
  'capitals_inside': False,
  'prefix-1': 'D',
  'prefix-1-lower': 'd',
  'prefix-2': 'Di',
  'prefix-2-lower': 'di',
  'prefix-3': 'Dit',
  'prefix-3-lower': 'dit',
  'suffix-1': 'n',
  'suffix-1-lower': 'n',
  'suffix-2': 'en',
  'suffix-2-lower': 'en',
  'suffix-3': 'jen',
  'suffix-3-lower': 'jen',
  'lowercase_word': 'ditjen',
  'prev_word': '',
  'next_word': 'Bea',
  'prev_word_is_capitalized': False,
  'next_word_is_capitalized': True,
  '2-prev-word': '',
  '2-next-word': 'dan'},
 {'word': 'Bea',
  'is_first': False,
  'is_last': False,
  'is_capitalized': True,
  'is_all_caps': False,
  'is_all_lower': False,
  'has_hyphen': False,
  'is_numeric': False,
  'capitals_inside': False,
  'prefix-1': 'B',
  'prefix-1-lower': 'b',
  'prefix-2': 'Be',
  'prefix-2-lower': 'be',
  'prefix-3': 'Bea',
  'pref

In [15]:
train(sentences, tags)

In [17]:
def save_model(model_path):
    pickle_out = open(model_path, "wb")
    pickle.dump(clf, pickle_out)
    pickle_out.close()

In [18]:
model_path = "/content/postagger_model.pkl"
save_model(model_path)

In [19]:
def load_model(model_path):
    pickle_in = open(model_path, "rb")
    load_data = pickle.load(pickle_in)
    return load_data

In [20]:
model = load_model(model_path)

In [21]:
def get_pos_tag(text):
    result = []
    sents = nltk.sent_tokenize(text)
    symbols = ['!', '&', '(', ')', '*', '?', ',', '.', '<', '>', '/', ':', ';',
                '[', ']', '\\', '^', '`', '{', '}', '|', '~', '"', '“', "'"]
    for sent in sents:
        tokenized_word = tokenizer.tokenize(sent)
        if sent:
            tags = model.predict(
                [
                    features(tokenized_word, index)
                    for index in range(len(tokenized_word))
                ]
            )
            for i in range(len(tags)):
                if tokenized_word[i] in symbols:
                    result.append((tokenized_word[i], "SYM"))
                else:
                    result.append((tokenized_word[i], tags[i]))
    return result

In [22]:
text = "Lionel Messi pergi ke pasar di daerah Jakarta Pusat."

In [24]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [25]:
get_pos_tag(text)

[('Lionel', 'NNP'),
 ('Messi', 'NNP'),
 ('pergi', 'VB'),
 ('ke', 'IN'),
 ('pasar', 'NN'),
 ('di', 'IN'),
 ('daerah', 'NN'),
 ('Jakarta', 'NNP'),
 ('Pusat', 'NNP'),
 ('.', 'SYM')]

# NER

Pada pembuatan NER kali ini, Tool yang digunakan adalah Spacy

In [26]:
import pickle
import spacy
import random
from spacy.util import minibatch, compounding
from spacy import load, displacy
from spacy.training.example import Example

In [27]:
with open('/content/drive/MyDrive/Bootcamp AI/Dataset/dataset_ner_spacy.pickle', 'rb') as f:
    data = pickle.load(f)

In [28]:
nlp = spacy.blank("id")
nlp.add_pipe('ner')
nlp.begin_training()

In [29]:
ner = nlp.get_pipe("ner")

In [30]:
for _, annotations in data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        break

In [31]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [32]:
# training model
with nlp.disable_pipes(*unaffected_pipes):

  # training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(
            example,
            drop=0.5,  # dropout - make it harder to memorise data
            losses=losses,
            )

    print("Losses at iteration {}".format(iteration), losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Kebakaran terjadi di kawasan permukiman padat pend..." with entities "[[0, 9, 'ACCIDENT'], [58, 114, 'LOCATION'], [116, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" Presiden @ jokowi setelah divaksin tetap bekerja..." with entities "[[1, 9, 'JOBTITLE'], [10, 17, 'SOCIALMEDIA_ACCOUNT...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses at iteration 0 {'ner': 7879.820264174714}
Losses at iteration 1 {'ner': 5264.973710343014}
Losses at iteration 2 {'ner': 4757.083528360885}
Losses at iteration 3 {'ner': 4113.587501032051}
Losses at iteration 4 {'ner': 3962.484630674618}
Losses at iteration 5 {'ner': 3570.8432822188115}
Losses at iteration 6 {'ner': 3457.68453113647}
Losses at iteration 7 {'ner': 3268.4958266301765}
Losses at iteration 8 {'ner': 3094.8005632620634}
Losses at iteration 9 {'ner': 2921.6815461797178}
Losses at iteration 10 {'ner': 2820.33167926232}
Losses at iteration 11 {'ner': 2775.069217177894}
Losses at iteration 12 {'ner': 2595.767153599245}
Losses at iteration 13 {'ner': 2569.357325170651}
Losses at iteration 14 {'ner': 2536.5162668046523}
Losses at iteration 15 {'ner': 2446.6709467651294}
Losses at iteration 16 {'ner': 2324.143140280105}
Losses at iteration 17 {'ner': 2268.606589563962}
Losses at iteration 18 {'ner': 2297.7515509792975}
Losses at iteration 19 {'ner': 2257.3048791000556}
Loss

In [33]:
# test
doc = nlp("Lionel Messi pergi ke pasar di daerah Jakarta Pusat.")

print(doc.ents)
print("----")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

(Lionel Messi, Jakarta Pusat)
----
Entities [('Lionel Messi', 'PERSON'), ('Jakarta Pusat', 'LOCATION')]
